In [1]:
import requests

url = "https://doc.la.gov/about-the-dpsc/facility-locations/"

response = requests.get(url)

print("Status code:", response.status_code)
print("Page length:", len(response.text))


Status code: 200
Page length: 252061


In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")

facilities = soup.find_all("li", class_="location")

print("Number of facilities found:", len(facilities))


Number of facilities found: 124


In [3]:
first = facilities[0]

# Facility name
name = first.find("h3", class_="color-cool-grey").get_text(strip=True)

# Phone number (some facilities might not have one)
phone_tag = first.find("div", class_="phone-block")
if phone_tag:
    phone = phone_tag.find("h3").get_text(strip=True)
else:
    phone = None

# URL
url = first.find("a")["href"]

print("Facility Name:", name)
print("Phone:", phone)
print("URL:", url)


Facility Name: Allen Correctional Center
Phone: (337) 639-2943
URL: https://doc.la.gov/location/allen-correctional-center/


In [6]:
import pandas as pd

data = []

for facility in facilities:
    # Facility name
    name = facility.find("h3", class_="color-cool-grey").get_text(strip=True)
    
    # Phone (some may be missing)
    phone_tag = facility.find("div", class_="phone-block")
    if phone_tag:
        phone = phone_tag.find("h3").get_text(strip=True)
    else:
        phone = None
    
    # URL
    url = facility.find("a")["href"]
    
    data.append({
        "facility_name": name,
        "phone": phone,
        "url": url
    })

df = pd.DataFrame(data)

# Preview the first 5 rows
df.head(10)


,facility_name,phone,url
0,Allen Correctional Center,(337) 639-2943,https://doc.la.gov/location/allen-correctional...
1,B.B. Rayburn Correctional Center,(985) 661-6300,https://doc.la.gov/location/b-b-rayburn-correc...
2,David Wade Correctional Center,(318) 927-0400,https://doc.la.gov/location/david-wade-correct...
3,Dixon Correctional Institute,(225) 634-1200,https://doc.la.gov/location/dixon-correctional...
4,Elayn Hunt Correctional Center,(225) 642-3306,https://doc.la.gov/location/elayn-hunt-correct...
5,Louisiana Correctional Institute for Women,(225) 319-2701,https://doc.la.gov/location/louisiana-correcti...
6,Louisiana State Penitentiary,(225) 655-4411,https://doc.la.gov/location/louisiana-state-pe...
7,Raymond Laborde Correctional Center,(318) 876-2891,https://doc.la.gov/location/raymond-laborde-co...
8,Acadia Parish,None,https://doc.la.gov/location/acadia-parish-jail...
9,Allen Parish,None,https://doc.la.gov/location/allen-parish-jails...


In [9]:
import pandas as pd

data = []

# Each category block (like "Correctional Facilities", etc.)
category_blocks = soup.find_all("div", class_="location-list-block")

for block in category_blocks:
    # Get the category name
    category = block.find("h5", class_="category-name").get_text(strip=True)
    
    # Find all facilities inside this category
    facilities = block.find_all("li", class_="location")
    
    for facility in facilities:
        # Facility name
        name = facility.find("h3", class_="color-cool-grey").get_text(strip=True)
        
        # Phone (may be missing)
        phone_tag = facility.find("div", class_="phone-block")
        if phone_tag:
            phone = phone_tag.find("h3").get_text(strip=True)
        else:
            phone = None
        
        # URL
        url = facility.find("a")["href"]
        
        data.append({
            "facility_name": name,
            "phone": phone,
            "url": url,
            "category": category
        })

df = pd.DataFrame(data)

print("Total rows:", len(df))
df.tail(10)


Total rows: 124


,facility_name,phone,url,category
114,Richland Parish Detention Center (Males),(318) 248-3070,https://doc.la.gov/location/richland-parish-de...,Transitional Work Program Facilities
115,Southwest Workforce Development Transitional W...,(337) 786-3009 & (337) 786-3008,https://doc.la.gov/location/southwest-workforc...,Transitional Work Program Facilities
116,St. Mary Law Enforcement Center Transitional W...,(337) 836-5520,https://doc.la.gov/location/st-mary-law-enforc...,Transitional Work Program Facilities
117,Tangipahoa Parish Jail,(985) 748-3392,https://doc.la.gov/location/tangipahoa-parish-...,Transitional Work Program Facilities
118,Tensas Parish Detention Center Transitional Wo...,(318) 749-5810,https://doc.la.gov/location/tensas-parish-dete...,Transitional Work Program Facilities
119,Terrebonne Transitional Work Program (LAWF),(985) 917-5254,https://doc.la.gov/location/terrebonne-transit...,Transitional Work Program Facilities
120,Union Parish Detention Center Transitional Wor...,(318) 368-9827,https://doc.la.gov/location/union-parish-deten...,Transitional Work Program Facilities
121,W. Baton Rouge Parish Transitional Work Program*,(225) 387-3272,https://doc.la.gov/location/w-baton-rouge-pari...,Transitional Work Program Facilities
122,Webster – Bayou Dorcheat Correctional Center T...,(318) 371-3554,https://doc.la.gov/location/webster-bayou-dorc...,Transitional Work Program Facilities
123,West Feliciana Parish Transitional Work Progra...,(225) 635-3807,https://doc.la.gov/location/west-feliciana-par...,Transitional Work Program Facilities


In [10]:
output_file = "louisiana_doc_facilities.csv"
df.to_csv(output_file, index=False)

print("CSV saved as:", output_file)


CSV saved as: louisiana_doc_facilities.csv
